In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ================= USER CONFIGURATION =================
#TARGET_YEAR = "2018"
TARGET_YEARS = [2019, 2020]
USERNAME = "hansheng@openbits.io"
PASSWORD = "[7vid|L|"
DOWNLOAD_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Downloads_zips/"
LOGIN_URL = "https://freddiemac.embs.com/FLoan/secure/login.php?pagename=downloadA"
# ======================================================

def setup_driver(download_path):
    if not os.path.exists(download_path):
        os.makedirs(download_path, exist_ok=True)
    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": download_path,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    options.add_experimental_option("detach", True)
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

def main():
    driver = setup_driver(DOWNLOAD_DIR)

    try:
        # 1. Login Phase
        driver.get(LOGIN_URL)
        try:
            driver.find_element(By.NAME, "username").send_keys(USERNAME)
            driver.find_element(By.NAME, "password").send_keys(PASSWORD)
        except:
            pass

        print("\n" + "!"*50)
        print("ACTION REQUIRED: Solve CAPTCHA and click LOGIN.")
        print("!"*50 + "\n")

        # Wait up to 10 mins for the URL to indicate we are logged in
        WebDriverWait(driver, 600).until(EC.url_contains("downloadA.php"))
        print(">> Login URL detected. Checking for content...")

        # 2. TERMS OF USE CHECKER (The "1 Link" Fix)
        # We wait up to 10 seconds to see if an "Accept" button exists
        try:
            accept_btn = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//input[@value='Accept' or @value='I Agree' or @value='Continue']"))
            )
            print(">> Terms of Use detected. Clicking 'Accept'...")
            accept_btn.click()
            time.sleep(3) # Wait for reload
        except:
            print(">> No 'Accept' button found. Proceeding...")

        # 3. WAIT FOR TABLE DATA
        # This is the crucial fix: Don't scan until we see "historical_data"
        print(">> Waiting for file list to load...")
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "historical_data"))
            )
            print(">> File list loaded successfully!")
        except:
            print(">> Warning: Timed out waiting for 'historical_data' links. Scanning page anyway...")

        # 4. FIND THE 2017 LINK
        # We search specifically for the year in the link text

        for year in TARGET_YEARS:
            year_str = str(year)
            links = driver.find_elements(By.TAG_NAME, "a")

            for link in links:
                text = link.text
                href = link.get_attribute("href")
            # Match if '2017' is in the text AND it's a zip file
                if text and year_str in text and "zip" in text.lower():
                    target_link = link
                    break
            # Fallback: check href if text is empty
                if href and year_str in href and ".zip" in href:
                    target_link = link
                    break

            if target_link:
                filename = target_link.text if target_link.text else target_link.get_attribute("href").split("/")[-1]
                print(f">> FOUND: {filename}")
                print(">> Clicking download...")
            
            # Scroll to it and click
                driver.execute_script("arguments[0].scrollIntoView();", target_link)
                time.sleep(1)
                target_link.click()


            # 5. MONITOR DOWNLOAD
            print(">> Download started. Monitoring folder...")
            while True:
                files = os.listdir(DOWNLOAD_DIR)
                if any(f.endswith(".crdownload") for f in files):
                    time.sleep(2)
                elif any(year_str in f for f in files):
                    print(f">> SUCCESS: {year_str} file found in folder!")
                    break
                else:
                    time.sleep(2)
        else:
            print(f">> ERROR: No {year_str} zip file found on page.")
            print(">> Debug: Here are the first 10 links I see:")
            for l in links[:10]:
                print(f"   - Text: {l.text} | Href: {l.get_attribute('href')}")

    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

: 

In [5]:
import os
import zipfile
import glob

# ================= USER CONFIGURATION =================
# 1. SOURCE: Where are the zip files now?
SOURCE_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Downloads_zips/"

# 2. TARGET: Where do you want the raw files?
# (I added a new folder name "Extracted_Data" to keep it organized)
TARGET_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/"
# ======================================================

def ensure_dir(directory):
    if not os.path.exists(directory):
        try:
            os.makedirs(directory)
        except PermissionError:
            print(f"\nCRITICAL ERROR: Permission Denied creating {directory}")
            print("You typically need to run 'sudo chown' on Linux external drives.")
            print(f"Try running this in terminal: sudo chown -R $USER {os.path.dirname(directory)}")
            exit(1)

def recursive_unzip(source, target):
    ensure_dir(target)
    
    # --- PHASE 1: Extract Main Zips from Source to Target ---
    zip_files = glob.glob(os.path.join(source, "*.zip"))
    
    if not zip_files:
        print(f"No .zip files found in {source}")
        return

    print(f"Found {len(zip_files)} top-level zip files.")
    
    for zip_path in zip_files:
        filename = os.path.basename(zip_path)
        print(f">> Extracting Main File: {filename} ...")
        try:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(target)
        except zipfile.BadZipFile:
            print(f"   [Error] Bad Zip File: {filename}")
        except Exception as e:
            print(f"   [Error] Failed to extract {filename}: {e}")

    # --- PHASE 2: Handle Nested Zips (The "Lowest Level" part) ---
    # We loop until no .zip files remain in the target folder
    while True:
        # Find zips inside the target folder (e.g., 2017Q1.zip extracted from 2017.zip)
        nested_zips = []
        for root, dirs, files in os.walk(target):
            for file in files:
                if file.endswith(".zip"):
                    nested_zips.append(os.path.join(root, file))
        
        if not nested_zips:
            break # No more zips found, we are done!

        print(f"\n>> Found {len(nested_zips)} internal zip files. Extracting deeper...")
        
        for nested_zip_path in nested_zips:
            try:
                # Extract contents to the same folder where the zip sits
                parent_folder = os.path.dirname(nested_zip_path)
                
                with zipfile.ZipFile(nested_zip_path, 'r') as zip_ref:
                    zip_ref.extractall(parent_folder)
                
                # DELETE the intermediate zip to save space and clean up
                os.remove(nested_zip_path)
                print(f"   Converted: {os.path.basename(nested_zip_path)}")
                
            except zipfile.BadZipFile:
                print(f"   [Warning] Corrupt nested zip: {nested_zip_path}")
            except Exception as e:
                print(f"   [Error] Could not process {nested_zip_path}: {e}")

    print("\n" + "="*50)
    print("DONE! All files extracted to lowest level.")
    print(f"Location: {TARGET_DIR}")
    print("="*50)

if __name__ == "__main__":
    recursive_unzip(SOURCE_DIR, TARGET_DIR)

Found 4 top-level zip files.
>> Extracting Main File: historical_data_2020.zip ...
>> Extracting Main File: historical_data_2018.zip ...
>> Extracting Main File: historical_data_2019.zip ...
>> Extracting Main File: historical_data_2017.zip ...

>> Found 16 internal zip files. Extracting deeper...
   Converted: historical_data_2018Q1.zip
   Converted: historical_data_2017Q4.zip
   Converted: historical_data_2020Q1.zip
   Converted: historical_data_2017Q3.zip
   Converted: historical_data_2018Q2.zip
   Converted: historical_data_2020Q4.zip
   Converted: historical_data_2020Q2.zip
   Converted: historical_data_2019Q3.zip
   Converted: historical_data_2018Q3.zip
   Converted: historical_data_2019Q2.zip
   Converted: historical_data_2018Q4.zip
   Converted: historical_data_2017Q1.zip
   Converted: historical_data_2019Q4.zip
   Converted: historical_data_2019Q1.zip
   Converted: historical_data_2017Q2.zip
   Converted: historical_data_2020Q3.zip

DONE! All files extracted to lowest level.
L

In [ ]:
orig_columns = ["Credit_Score","First_Payment_Date","First_Time_Home_Buyer","Maturity_Date","MSA","Insurance_pct","Units","Occupancy",
                "Combined_LTV","DTI","UPB","LTV","Interest","Channel","PPM","Product_Type","State","Property_Type","ZIP","ID","Loan_Purpose",
                "Term","Borrowers","Seller","Servicer","Super_Confirming","Program"]

perf_columns = ["ID","Date","C_UPB","Del_Status",'Loan_Age',"Remain_Age","Repurchase","Modification","Zero_UPB","Zero_UPB_Date","C_IR",
                "C_deferred_UPB","DDLPI","MI_Rec","Net_Sales","Non_MI_Rec","Expenses","Legal_Costs","Maintain_Cost","Taxes","Other_Expenses",
                "Actual_Loss","Modification_Cost","Step_Modification","Deferred_PP","ELTV","Zero_UPB_Removal","Del_Accrued_Interest","Del_Disaster","Borrower_Assistance"]

#historical_data_Q12017.txt is the origination data of mortgages originated in Q12017,  historical_data_time_Q12017.txt is the performance data, for 2017Q1 to 2020Q4 randomly sample 5000 mortgages and combine into one file 
# TARGET_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/" 
ORIG_COLUMNS = [
    "CREDIT_SCORE", "FIRST_PAYMENT_DATE", "FIRST_TIME_HOMEBUYER_FLAG", "MATURITY_DATE",
    "MSA", "MORTGAGE_INSURANCE_PERCENTAGE", "NUMBER_OF_UNITS", "OCCUPANCY_STATUS",
    "ORIGINAL_CLTV", "ORIGINAL_DEBT_TO_INCOME_RATIO", "ORIGINAL_UPB", "ORIGINAL_LTV",
    "ORIGINAL_INTEREST_RATE", "CHANNEL", "PREPAYMENT_PENALTY_MORTGAGE_FLAG",
    "AMORTIZATION_TYPE", "PROPERTY_STATE", "PROPERTY_TYPE", "POSTAL_CODE",
    "LOAN_SEQUENCE_NUMBER", "LOAN_PURPOSE", "ORIGINAL_LOAN_TERM", "NUMBER_OF_BORROWERS",
    "SELLER_NAME", "SERVICER_NAME", "SUPER_CONFORMING_FLAG", "PRE_HARP_LOAN_SEQUENCE_NUMBER",
    "PROGRAM_INDICATOR", "HARP_INDICATOR", "PROPERTY_VALUATION_METHOD", "INTEREST_ONLY_INDICATOR"
] 

In [3]:
import pandas as pd
import os
import glob

# ---------------- CONFIGURATION ---------------- #
TARGET_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/"
OUTPUT_FILE = os.path.join(TARGET_DIR, "combined_sampled_mortgages_2017_2020.csv")
SAMPLE_SIZE = 5000
YEARS = [2017, 2018, 2019, 2020]
QUARTERS = [1, 2, 3, 4]

# Freddie Mac Standard Origination Column Names (Common Schema)
# If your files have headers, change header=None to header=0 in the read_csv call
ORIG_COLUMNS = [
    "CREDIT_SCORE", "FIRST_PAYMENT_DATE", "FIRST_TIME_HOMEBUYER_FLAG", "MATURITY_DATE",
    "MSA", "MORTGAGE_INSURANCE_PERCENTAGE", "NUMBER_OF_UNITS", "OCCUPANCY_STATUS",
    "ORIGINAL_CLTV", "ORIGINAL_DEBT_TO_INCOME_RATIO", "ORIGINAL_UPB", "ORIGINAL_LTV",
    "ORIGINAL_INTEREST_RATE", "CHANNEL", "PREPAYMENT_PENALTY_MORTGAGE_FLAG",
    "AMORTIZATION_TYPE", "PROPERTY_STATE", "PROPERTY_TYPE", "POSTAL_CODE",
    "LOAN_SEQUENCE_NUMBER", "LOAN_PURPOSE", "ORIGINAL_LOAN_TERM", "NUMBER_OF_BORROWERS",
    "SELLER_NAME", "SERVICER_NAME", "SUPER_CONFORMING_FLAG", "PRE_HARP_LOAN_SEQUENCE_NUMBER",
    "PROGRAM_INDICATOR", "HARP_INDICATOR", "PROPERTY_VALUATION_METHOD", "INTEREST_ONLY_INDICATOR"
]
# ----------------------------------------------- #

def main():
    all_sampled_data = []

    print(f"Starting processing in: {TARGET_DIR}")

    for year in YEARS:
        for q in QUARTERS:
            # Construct the filename pattern based on your example: historical_data_Q12017.txt
            file_suffix = f"{year}Q{q}"
            filename = f"historical_data_{file_suffix}.txt"
            file_path = os.path.join(TARGET_DIR, filename)

            if os.path.exists(file_path):
                print(f"Processing {filename}...")
                
                try:
                    # Read the origination data
                    # separator is usually pipe '|' for Freddie Mac, sometimes whitespace or comma.
                    # Adjust sep='|' if your file is pipe-delimited.
                    df = pd.read_csv(
                        file_path, 
                        sep='|', 
                        header=None, 
                        names=ORIG_COLUMNS,
                        index_col=False,
                        low_memory=False
                    )

                    # Check if enough data exists to sample
                    if len(df) >= SAMPLE_SIZE:
                        sampled_df = df.sample(n=SAMPLE_SIZE, random_state=42)
                    else:
                        print(f"Warning: {filename} has fewer than {SAMPLE_SIZE} rows. Taking all.")
                        sampled_df = df

                    # Add a column to track which quarter this came from (optional but helpful)
                    sampled_df['SOURCE_QUARTER'] = file_suffix
                    
                    all_sampled_data.append(sampled_df)

                except Exception as e:
                    print(f"Error reading {filename}: {e}")
            else:
                print(f"File not found: {filename} - Skipping.")

    # Combine all dataframes
    if all_sampled_data:
        print("Concatenating data...")
        final_df = pd.concat(all_sampled_data, ignore_index=True)
        
        print(f"Saving combined file to {OUTPUT_FILE}...")
        final_df.to_csv(OUTPUT_FILE, index=False, sep='|')
        
        print("Done!")
        print(f"Total mortgages sampled: {len(final_df)}")
    else:
        print("No data was collected.")

if __name__ == "__main__":
    main()

Starting processing in: /media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/
Processing historical_data_2017Q1.txt...


/tmp/ipykernel_4235/1711330823.py:45: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


Processing historical_data_2017Q2.txt...


/tmp/ipykernel_4235/1711330823.py:45: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


Processing historical_data_2017Q3.txt...


...

Processing historical_data_2020Q3.txt...


/tmp/ipykernel_4235/1711330823.py:45: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


Processing historical_data_2020Q4.txt...


/tmp/ipykernel_4235/1711330823.py:45: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(


Concatenating data...
Saving combined file to /media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/combined_sampled_mortgages_2017_2020.csv...
Done!
Total mortgages sampled: 80000


In [4]:
import pandas as pd
import os

# ---------------- CONFIGURATION ---------------- #
TARGET_DIR = "/media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/"
INPUT_SAMPLED_FILE = os.path.join(TARGET_DIR, "combined_sampled_mortgages_2017_2020.csv")
OUTPUT_PERFORMANCE_FILE = os.path.join(TARGET_DIR, "combined_performance_history_2017_2020.csv")

CHUNK_SIZE = 100000  # Reads 100k rows at a time to prevent crashing

# Standard Freddie Mac Performance Columns
TIME_COLUMNS = [
    "LOAN_SEQUENCE_NUMBER", "MONTHLY_REPORTING_PERIOD", "CURRENT_ACTUAL_UPB", 
    "CURRENT_LOAN_DELINQUENCY_STATUS", "LOAN_AGE", "REMAINING_MONTHS_TO_LEGAL_MATURITY", 
    "REPURCHASE_FLAG", "MODIFICATION_FLAG", "ZERO_BALANCE_CODE", "ZERO_BALANCE_EFFECTIVE_DATE", 
    "CURRENT_INTEREST_RATE", "CURRENT_DEFERRED_UPB", "DUE_DATE_OF_LAST_PAID_INSTALLMENT", 
    "MI_RECOVERIES", "NET_SALES_PROCEEDS", "NON_MI_RECOVERIES", "EXPENSES", "LEGAL_COSTS", 
    "MAINTENANCE_AND_PRESERVATION_COSTS", "TAXES_AND_INSURANCE", "MISC_EXPENSES", 
    "ACTUAL_LOSS_CALCULATION", "MODIFICATION_COST", "STEP_MODIFICATION_FLAG", 
    "DEFERRED_PAYMENT_PLAN", "ESTIMATED_LOAN_TO_VALUE", "ZERO_BALANCE_REMOVAL_UPB", 
    "DELINQUENT_ACCRUED_INTEREST", "DELINQUENCY_DUE_TO_DISASTER", 
    "BORROWER_ASSISTANCE_STATUS_CODE", "CURRENT_MONTH_MODIFICATION_COST", "INTEREST_BEARING_UPB"
]
# ----------------------------------------------- #

def main():
    print(f"Loading lookup list from: {INPUT_SAMPLED_FILE}")
    
    # 1. Load the sampled origination data to get the IDs
    try:
        df_lookup = pd.read_csv(INPUT_SAMPLED_FILE, sep='|', low_memory=False, dtype={'LOAN_SEQUENCE_NUMBER': str})
    except FileNotFoundError:
        print("Error: Input file not found. Please run the sampling script first.")
        return

    # 2. Get unique Quarters to process
    # We use the 'SOURCE_QUARTER' column we created in the previous step (e.g., "Q12017")
    if 'SOURCE_QUARTER' not in df_lookup.columns:
        print("Error: 'SOURCE_QUARTER' column missing. The script relies on this to find the correct time file.")
        return

    unique_quarters = df_lookup['SOURCE_QUARTER'].unique()
    print(f"Found {len(df_lookup)} mortgages across {len(unique_quarters)} quarters.")

    all_performance_data = []

    # 3. Iterate through each quarter present in the file
    for quarter_id in unique_quarters:
        
        # Get the IDs specific to this quarter
        quarter_subset = df_lookup[df_lookup['SOURCE_QUARTER'] == quarter_id]
        target_ids = set(quarter_subset['LOAN_SEQUENCE_NUMBER'])
        
        # Construct the Time Filename
        # NOTE: Adjust this pattern if your files are named "2017Q1" instead of "Q12017"
        # Based on your prompt, we look for: historical_data_time_Q12017.txt
        time_filename = f"historical_data_time_{quarter_id}.txt"
        time_path = os.path.join(TARGET_DIR, time_filename)

        if os.path.exists(time_path):
            print(f"Scanning {time_filename} for {len(target_ids)} Loan IDs...")
            
            try:
                # Read in chunks
                chunk_iterator = pd.read_csv(
                    time_path, 
                    sep='|', 
                    header=None, 
                    names=TIME_COLUMNS,
                    index_col=False, 
                    dtype={'LOAN_SEQUENCE_NUMBER': str}, 
                    chunksize=CHUNK_SIZE, 
                    low_memory=False
                )

                rows_found = 0
                for chunk in chunk_iterator:
                    # Filter: Keep row ONLY if Loan ID is in our target list
                    filtered_chunk = chunk[chunk['LOAN_SEQUENCE_NUMBER'].isin(target_ids)]
                    
                    if not filtered_chunk.empty:
                        # Append source quarter just in case
                        filtered_chunk = filtered_chunk.copy()
                        filtered_chunk['SOURCE_QUARTER'] = quarter_id
                        
                        all_performance_data.append(filtered_chunk)
                        rows_found += len(filtered_chunk)

                print(f"  -> Extracted {rows_found} performance records.")

            except Exception as e:
                print(f"  Error reading {time_filename}: {e}")

        else:
            print(f"Warning: Time file not found: {time_filename}")

    # 4. Combine and Save
    if all_performance_data:
        print("\nConcatenating all performance history...")
        final_df = pd.concat(all_performance_data, ignore_index=True)
        
        print(f"Saving to {OUTPUT_PERFORMANCE_FILE}...")
        final_df.to_csv(OUTPUT_PERFORMANCE_FILE, index=False, sep='|')
        print(f"Done! Total rows: {len(final_df)}")
    else:
        print("No performance data found.")

if __name__ == "__main__":
    main()

Loading lookup list from: /media/hansheng/cc7df9bc-e728-4b8d-a215-b64f31876acc/cdo-tee-mock/prepayment/data/Extracted_data/combined_sampled_mortgages_2017_2020.csv
Found 80000 mortgages across 16 quarters.
Scanning historical_data_time_2017Q1.txt for 5000 Loan IDs...
  -> Extracted 281985 performance records.
Scanning historical_data_time_2017Q2.txt for 5000 Loan IDs...
  -> Extracted 266333 performance records.
Scanning historical_data_time_2017Q3.txt for 5000 Loan IDs...
  -> Extracted 269531 performance records.
Scanning historical_data_time_2017Q4.txt for 5000 Loan IDs...
  -> Extracted 257471 performance records.
Scanning historical_data_time_2018Q1.txt for 5000 Loan IDs...
  -> Extracted 230767 performance records.
Scanning historical_data_time_2018Q2.txt for 5000 Loan IDs...
  -> Extracted 200383 performance records.
Scanning historical_data_time_2018Q3.txt for 5000 Loan IDs...
  -> Extracted 189793 performance records.
Scanning historical_data_time_2018Q4.txt for 5000 Loan IDs.